#**A03. Word Sense**

In [ ]:
import nltk
nltk.download('all')

#**Problem1**

##Use sorted() and set() to get a sorted list of tags used in the Brown corpus, removing duplicates.

In [ ]:
print(sorted(set([tag for (word, tag) in nltk.corpus.brown.tagged_words()])))

["'", "''", '(', '(-HL', ')', ')-HL', '*', '*-HL', '*-NC', '*-TL', ',', ',-HL', ',-NC', ',-TL', '--', '---HL', '.', '.-HL', '.-NC', '.-TL', ':', ':-HL', ':-TL', 'ABL', 'ABN', 'ABN-HL', 'ABN-NC', 'ABN-TL', 'ABX', 'AP', 'AP$', 'AP+AP-NC', 'AP-HL', 'AP-NC', 'AP-TL', 'AT', 'AT-HL', 'AT-NC', 'AT-TL', 'AT-TL-HL', 'BE', 'BE-HL', 'BE-TL', 'BED', 'BED*', 'BED-NC', 'BEDZ', 'BEDZ*', 'BEDZ-HL', 'BEDZ-NC', 'BEG', 'BEM', 'BEM*', 'BEM-NC', 'BEN', 'BEN-TL', 'BER', 'BER*', 'BER*-NC', 'BER-HL', 'BER-NC', 'BER-TL', 'BEZ', 'BEZ*', 'BEZ-HL', 'BEZ-NC', 'BEZ-TL', 'CC', 'CC-HL', 'CC-NC', 'CC-TL', 'CC-TL-HL', 'CD', 'CD$', 'CD-HL', 'CD-NC', 'CD-TL', 'CD-TL-HL', 'CS', 'CS-HL', 'CS-NC', 'CS-TL', 'DO', 'DO*', 'DO*-HL', 'DO+PPSS', 'DO-HL', 'DO-NC', 'DO-TL', 'DOD', 'DOD*', 'DOD*-TL', 'DOD-NC', 'DOZ', 'DOZ*', 'DOZ*-TL', 'DOZ-HL', 'DOZ-TL', 'DT', 'DT$', 'DT+BEZ', 'DT+BEZ-NC', 'DT+MD', 'DT-HL', 'DT-NC', 'DT-TL', 'DTI', 'DTI-HL', 'DTI-TL', 'DTS', 'DTS+BEZ', 'DTS-HL', 'DTX', 'EX', 'EX+BEZ', 'EX+HVD', 'EX+HVZ', 'EX+MD', '

#**Problem 2**

##Recall the example of a bigram tagger which encountered a word it hadn't seen during training, and tagged the rest of the sentence as None. 

##It is possible for a bigram tagger to fail part way through a sentence even if it contains no unseen words (even if the sentence was used during training). 

##In what circumstance can this happen? Can you write a program to find some examples of this?

タグ付けの際に、学習対象の単語であっても、学習時と異なるコンテキストで与えられた場合には、その単語は認識されずBigram Taggerはそれ以降の単語の認識にも連鎖的に失敗します。

このようにBigram Taggerには、一度単語を認識できなかったとき、それ以降の単語は連鎖的に認識されなくなるという性質があります。

よって、タグのペアに認識されなかった単語(unknown)を含むとき、残りのタグのペアは直前の認識されなかったタグのペアをもとに推測されるので、結果的に問題文で指摘された状況が起こります。

In [ ]:
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories='news')

training_set = brown_tagged_sents[0]
training_sentence = brown.sents()[0]

shuffled_sentence = sorted(training_sentence)

bigram_training = list(nltk.bigrams(shuffled_sentence))
bigram_shuffled = list(nltk.bigrams(training_sentence))

# Program to find examples of this would pull out all the extant bigrams 
# -containing a particular word in the training set. And then it would do the same for the testing set. 
# Subtract the training set from the testing set. 
# If anything remains, it is a bigram context that is not accounted for.

def test_unknown_contexts(corpus_one, corpus_two):
    #tests to see if two inputted sentences are identical
    if list(set(corpus_one)-set(corpus_two)):
        print("There are unknown contexts")
    else:
        print("All contexts are known")

test_unknown_contexts(bigram_training, bigram_shuffled)

There are unknown contexts


In [ ]:
# 問題２の解答

from nltk.corpus import brown
from random import shuffle
from nltk.tag import BigramTagger

brown_tagged_sents = brown.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown.sents(categories='news')

bigram_tagger = nltk.BigramTagger(train_sents)      # training process

print("同じコンテキスト" + str(bigram_tagger.tag(test_sents[2007])))          # the words seen during training comes with same contexts
print("違うコンテキスト" + str(bigram_tagger.tag(sorted(test_sents[2007]))))  # the words seen during training comes with different context
#　異なるコンテキスト（ソートを用いた）で単語を与えた場合、連鎖的に単語の認識に失敗している

同じコンテキスト[('Various', 'JJ'), ('of', 'IN'), ('the', 'AT'), ('apartments', 'NNS'), ('are', 'BER'), ('of', 'IN'), ('the', 'AT'), ('terrace', 'NN'), ('type', 'NN'), (',', ','), ('being', 'BEG'), ('on', 'IN'), ('the', 'AT'), ('ground', 'NN'), ('floor', 'NN'), ('so', 'CS'), ('that', 'CS'), ('entrance', 'NN'), ('is', 'BEZ'), ('direct', 'JJ'), ('.', '.')]
違うコンテキスト[(',', ','), ('.', None), ('Various', None), ('apartments', None), ('are', None), ('being', None), ('direct', None), ('entrance', None), ('floor', None), ('ground', None), ('is', None), ('of', None), ('of', None), ('on', None), ('so', None), ('terrace', None), ('that', None), ('the', None), ('the', None), ('the', None), ('type', None)]


#**Problem 3**

Preprocess the Brown News data by replacing low frequency words with UNK, but leaving the tags untouched. Now train and evaluate a bigram tagger on this data. How much does this help? What is the contribution of the unigram tagger and default tagger now?

unigram taggerとdefault taggerは、bigram taggerの精度を向上させるという点で非常に役立ちます。

bigram taggerがｎ番目とn-1番目の単語をタグ付けする場合、文の最初の単語がコンテキストを持たないことによって生じる問題は、unigram taggerとdefault taggerを補助的に用いることで解決されます。

In [ ]:
# 問題３の解答

from nltk.tag import BigramTagger
from nltk.corpus import brown
from nltk.probability import FreqDist

# 各単語の頻度表を取得する
brown_words = brown.words(categories='news')
fdist = FreqDist(word.lower() for word in brown_words)

brown_tagged_sents = brown.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)

# テスト用のデータを用意する
test_sents = brown_tagged_sents[:size]

# 学習用のデータを用意する
# 頻度の低い単語はUNKで置き換える
train_sents = []
for sent in brown_tagged_sents[:size]:
  train_sent = []
  for (word, tag) in sent:
    if fdist[word.lower()] <= 3:  # 頻度が３以下のときはUNKで置き換える 　頻度の値は適宜変更する
      word = 'UNK'
    train_sent.append((word, tag))
  train_sents.append(train_sent)

print(brown_tagged_sents[2005])
print(train_sents[2005])

t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
bigram_tagger = nltk.BigramTagger(train_sents, backoff=t1)

# Bigram Tagger 単体テスト用
#bigram_tagger = nltk.BigramTagger(train_sents)

bigram_tagger.evaluate(test_sents)

# Bigram Tagger 単体だと精度は20%前後になる
# 頻度が低い単語をUNKで置き換えた上で、Default Tagger と Unigram Tagger と Bigram Tagger を組み合わせると８０％前後の精度に落ち着く

[('Sixty-seven', 'CD'), ('living', 'VBG'), ('units', 'NNS'), ('are', 'BER'), ('being', 'BEG'), ('added', 'VBN'), ('to', 'IN'), ('the', 'AT'), ('165-unit', 'JJ'), ('Harbor', 'NN-TL'), ('View', 'NN-TL'), ('Apartments', 'NNS-TL'), ('in', 'IN'), ('the', 'AT'), ('Cherry', 'NN-TL'), ('Hill', 'NN-TL'), ('section', 'NN'), ('.', '.')]
[('UNK', 'CD'), ('living', 'VBG'), ('units', 'NNS'), ('are', 'BER'), ('being', 'BEG'), ('added', 'VBN'), ('to', 'IN'), ('the', 'AT'), ('UNK', 'JJ'), ('UNK', 'NN-TL'), ('View', 'NN-TL'), ('Apartments', 'NNS-TL'), ('in', 'IN'), ('the', 'AT'), ('UNK', 'NN-TL'), ('Hill', 'NN-TL'), ('section', 'NN'), ('.', '.')]


0.8583643574419195